In [1]:
# Import Libraries
import requests
from pprint import PrettyPrinter
pp = PrettyPrinter()
import pandas as pd
from dotenv import load_dotenv
import os
import helpers
load_dotenv()
api_key = os.getenv("TMDB_API_KEY")
omdb_api_key = os.getenv("OMDB_API_KEY")

Step 1 :Get List of all bollywood movies release afer 2010


In [2]:
def get_movie_list()->pd.DataFrame:
    url = f"https://api.themoviedb.org/3/discover/movie?api_key={api_key}&include_adult=false&include_video=false&sort_by=revenue.desc&with_original_language=hi"
    response = requests.get(url).json()
    total_pages = response['total_pages']
    movies_info = pd.DataFrame()
    for page in range(1,total_pages):
        page_url  = f"{url}&page={page}"
        page_response = requests.get(page_url).json()['results']
        movies_info =pd.concat([movies_info,pd.DataFrame(page_response)])

    movies_info = movies_info[(movies_info['release_date']>='2000-01-01') & (movies_info['release_date']<='2024-04-01')]
    columns_to_keep = ['title','genre_ids','id','popularity','release_date']
    movies_info = movies_info[columns_to_keep]
    return movies_info

In [3]:
movies_info = get_movie_list()
movies_info=movies_info.reset_index()
movies_info.shape

(4032, 6)

In [4]:
movies_info.head()

,index,title,genre_ids,id,popularity,release_date
0,0,Dangal,"[18, 10751, 35, 28]",360814,26.566,2016-12-21
1,1,Jab Tak Hai Jaan,"[18, 10749]",132316,10.747,2012-11-13
2,2,Zindagi Na Milegi Dobara,"[18, 35, 10749]",61202,18.269,2011-07-15
3,3,Bajrangi Bhaijaan,"[35, 18, 28]",348892,17.963,2015-07-17
4,4,Secret Superstar,"[18, 10402]",441889,9.825,2017-10-19


In [14]:
movie_details = pd.DataFrame()
for i in range(len(movies_info)):
    movie_details= pd.concat([movie_details,
                              pd.DataFrame([helpers.get_features_movie(movies_info['id'][i])]
                                           )])

In [15]:
movie_details.to_parquet('movie_details.parquet')

In [2]:
movie_details = pd.read_parquet('movie_details.parquet')

In [3]:
movie_details_filtered = movie_details[~((movie_details['revenue']==0) | (movie_details['budget']==0))]
movie_details_filtered.shape

(322, 10)

In [4]:
import hopsworks
api_key = os.getenv("HOPSWORKS_KEY")

project = hopsworks.login(api_key_value=api_key)

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/602262
Connected. Call `.close()` to terminate connection gracefully.


In [6]:
# Get or create the 'transactions_fraud_batch_fg' feature group
movies_fg = fs.get_or_create_feature_group(
    name="raw_movies_info",
    version=3,
    description="Information about bollywood movies released after 2001",
    primary_key=["id"]
)

# Insert data into feature group
movies_fg.insert(movie_details_filtered)

# Update feature descriptions
feature_descriptions = [
    {"name": "budget", "description": "Budget of movie"},
    {"name": "id", "description": "TMDB ID "},
    {"name": "imdb_id", "description": "IMDB ID"},
    {"name": "popularity", "description": "popularity of movie"},
    {"name": "release_date", "description": "release date of movie"},
    {"name": "revenue", "description": "total box office collection"},
    {"name": "title", "description": "Movie Name"},
    {"name": "cast_popularity", "description": "Sum of popularity of all actors"},
    {"name": "crew_popularity", "description": "sum of popularity of all crew members"},
    {"name": "star_present", "description": "whether movie has a bollywood star or not"}
]

for desc in feature_descriptions: 
    movies_fg.update_feature_description(desc["name"], desc["description"])

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/602262/fs/598085/fg/683393


Uploading Dataframe: 0.00% |          | Rows 0/322 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: raw_movies_info_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/602262/jobs/named/raw_movies_info_3_offline_fg_materialization/executions
